In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

In [2]:

db_config = {'user': 'practicum_student',         # username
             'pwd': 's65BlTKV3faNIGhmvJVzOqhs', # password
             'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
             'port': 6432,              # connection port
             'db': 'data-analyst-final-project-db'}          # the name of the database

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                                     db_config['pwd'],
                                                                       db_config['host'],
                                                                       db_config['port'],
                                                                       db_config['db'])

engine = create_engine(connection_string, connect_args={'sslmode':'require'})


In [3]:
with engine.connect() as con:
    f = con.execute("""SELECT * 
                    FROM books 
                    ORDER BY num_pages desc""")
    df_books = pd.DataFrame(f.fetchall())
    df_books.columns =f.keys()

    df_books

In [4]:
df_books.head()

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,298,236,Harry Potter Boxed Set Books 1-5 (Harry Potte...,2690,2004-09-13,253
1,684,354,The Complete Anne of Green Gables Boxed Set (A...,2088,1998-10-06,285
2,361,240,J.R.R. Tolkien 4-Book Boxed Set: The Hobbit an...,1728,2012-09-25,33
3,760,224,The Iliad/The Odyssey,1556,1999-11-01,217
4,685,57,The Complete Calvin and Hobbes,1456,2005-09-06,18


In [5]:
with engine.connect() as con:
    a = con.execute("""SELECT * 
                       FROM authors""")
    df_authors = pd.DataFrame(a.fetchall())
    df_authors.columns =a.keys()

In [6]:
df_authors.head()

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [7]:
with engine.connect() as con:
    p = con.execute("""select * 
                        from publishers""" )
    df_pub = pd.DataFrame(p.fetchall())
    df_pub.columns =p.keys()

In [8]:
df_pub.head()

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [9]:
with engine.connect() as con:
    r = con.execute("""SELECT *
                       FROM ratings""" )
    df_rate = pd.DataFrame(r.fetchall())
    df_rate.columns =r.keys()

In [10]:
df_rate.head()

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [11]:
with engine.connect() as con:
    re = con.execute("""SELECT * 
                     FROM reviews """)
    df_rev = pd.DataFrame(re.fetchall())
    df_rev.columns =re.keys()

In [12]:
df_rev.head()

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


1. Find the number of books released after January 1, 2000.

In [13]:
with engine.connect() as con:
    bn = con.execute("""SELECT count(book_id) 
                     FROM books 
                     WHERE publication_date > '2000-01-01' """)
    df2 = pd.DataFrame(bn.fetchall())
    df2.columns =bn.keys() 
    

In [14]:
df2

,count
0,819


2. Find the number of user reviews and the average rating for each book.

In [15]:
with engine.connect() as con:
    rr = con.execute("""SELECT re.book_id, count(DISTINCT review_id) as user_reviews, AVG(rating) as avg_ratings 
                     FROM ratings as r 
                     JOIN reviews as re 
                     ON r.book_id=re.book_id 
                     GROUP BY re.book_id""")
    df3 = pd.DataFrame(rr.fetchall())
    df3.columns =rr.keys() 
    

In [16]:
df3

,book_id,user_reviews,avg_ratings
0,1,2,3.6666666666666667
1,2,1,2.5000000000000000
2,3,3,4.6666666666666667
3,4,2,4.5000000000000000
4,5,4,4.0000000000000000
...,...,...,...
989,996,3,3.6666666666666667
990,997,3,3.4000000000000000
991,998,4,3.2000000000000000
992,999,2,4.5000000000000000


3. Identify the publisher that has released the greatest number of books with more than 50 pages (this will help you exclude brochures and similar publications from your analysis).

In [17]:
with engine.connect() as con:
    pb = con.execute("""
                     SELECT publisher, count(book_id) as book_num
                     FROM publishers as p  
                     JOIN books as b 
                         ON p.publisher_id = b.publisher_id 
                     WHERE num_pages > 50 
                     GROUP BY p.publisher 
                     ORDER BY count(book_id) desc LIMIT 5 """)
    df4 = pd.DataFrame(pb.fetchall())
    df4.columns =pb.keys() 
    

In [18]:
df4

,publisher,book_num
0,Penguin Books,42
1,Vintage,31
2,Grand Central Publishing,25
3,Penguin Classics,24
4,Bantam,19


4. Identify the author with the highest average book rating (look only at books with at least 50 ratings).

In [19]:
with engine.connect() as con:
    avb = con.execute("""SELECT author, avg(rating) 
                      FROM authors as a join books as b 
                      ON a.author_id = b.author_id 
                      JOIN ratings as r 
                      ON b.book_id = r.book_id 
                      GROUP BY author 
                      HAVING count(rating_id) >= 50 order by avg(rating) desc""")
    df5 = pd.DataFrame(avb.fetchall())
    df5.columns =avb.keys() 
    

In [20]:
df5

,author,avg
0,Diana Gabaldon,4.3000000000000000
1,J.K. Rowling/Mary GrandPré,4.2884615384615385
2,Agatha Christie,4.2830188679245283
3,Markus Zusak/Cao Xuân Việt Khương,4.2641509433962264
4,J.R.R. Tolkien,4.2409638554216867
5,Roald Dahl/Quentin Blake,4.2096774193548387
6,Louisa May Alcott,4.2037037037037037
7,Rick Riordan,4.1309523809523810
8,Arthur Golden,4.1071428571428571
9,Stephen King,4.0094339622641509


5. Find the average number of text reviews among users who rated more than 50 books.

In [23]:
with engine.connect() as con:
    rv = con.execute("""SELECT avg(Text_review) AS avg_Treview
                     FROM (SELECT username, Count(text) As Text_review 
                     FROM reviews 
                     GROUP BY username) as c_table 
                     
                     WHERE username IN (SELECT username FROM ratings GROUP BY username HAVING count(username) > 50)""")
    df6 = pd.DataFrame(rv.fetchall())
    df6.columns =rv.keys() 

In [24]:
df6

,avg_treview
0,24.3333333333333333
